<a href="https://colab.research.google.com/github/Luca4Spreafico/CHALLENGE-2---Ibuprofen/blob/main/Luca_Ibuprofen2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🌐 **Kaggle setup**

In [ ]:
working_dir = "/kaggle/working"
input_dir = "/kaggle/input"
dataset_dir = "/kaggle/input"

## 🌐 **Google Drive setup**

In [1]:
# ___________________________________________________________________________________ collegamento colab
from google.colab import drive

drive.mount("/gdrive")
working_dir = "/gdrive/My Drive/B University/Artificial Networks/an2dl2526c2"
input_dir = working_dir

%cd $working_dir

print(f"Working at folder: {working_dir}")
print(f"Input at folder: {input_dir}")

Mounted at /gdrive
/gdrive/My Drive/B University/Artificial Networks/an2dl2526c2
Working at folder: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2
Input at folder: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2


# Set up

In [2]:
# ___________________________________________________________________________________ Libraries

# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader

# Configurazione di TensorBoard e directory
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import copy
import shutil
from itertools import product
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.gridspec as gridspec

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

PyTorch version: 2.9.0+cu126
Device: cuda


Data Loading

In [3]:
# _________________________________________  DATA LOADING

dataset_dir = input_dir   # adjust to your real folder

train_data_dir = os.path.join(dataset_dir, "train_data")
test_data_dir  = os.path.join(dataset_dir, "test_data")
train_labels_path = os.path.join(dataset_dir, "train_labels.csv")


# ------------------ Check directories ------------------
if not os.path.exists(train_data_dir):
    print(f"Error: Training data folder not found at: {train_data_dir}")
else:
    print(f"Training folder found: {train_data_dir}")

if not os.path.exists(test_data_dir):
    print(f"Error: Test data folder not found at: {test_data_dir}")
else:
    print(f"Test folder found: {test_data_dir}")

# ------------------ Check labels file ------------------
if not os.path.exists(train_labels_path):
    print(f"Error: train_labels.csv not found at: {train_labels_path}")
else:
    print(f"Labels file found: {train_labels_path}")

# ------------------ Load labels ------------------
try:
    df_labels = pd.read_csv(train_labels_path)
    print(f"Loaded {len(df_labels)} training labels.")
except Exception as e:
    print(f"Error loading labels: {e}")

# ------------------ Load training images & masks ------------------
print("Loading training images and masks…")

file_list = sorted(os.listdir(train_data_dir))
N = len(df_labels)  # should be 1412

if len(file_list) != 2 * N:
    print(f"WARNING: Expected {2*N} files, found {len(file_list)}.")

# First half = images
image_files = file_list[:N]

# Second half = masks
mask_files = file_list[N:N*2]

train_images = []
train_masks = []
train_targets = []

for idx, row in df_labels.iterrows():
    filename_img  = image_files[idx]
    filename_mask = mask_files[idx]
    label         = row["label"]

    img_path  = os.path.join(train_data_dir, filename_img)
    mask_path = os.path.join(train_data_dir, filename_mask)

    try:
        img  = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        train_images.append(img)
        train_masks.append(mask)
        train_targets.append(label)

    except Exception as e:
        print(f"ERROR loading sample index {idx}: {e}")

print(f"Loaded {len(train_images)} training samples successfully.")



Training folder found: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2/train_data
Test folder found: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2/test_data
Labels file found: /gdrive/My Drive/B University/Artificial Networks/an2dl2526c2/train_labels.csv
Loaded 1412 training labels.
Loading training images and masks…
Loaded 1412 training samples successfully.


Exploration Dataset

In [5]:
print("\n===== DATASET EXPLORATION =====")

print("Training images:", len(train_images))
print("Training masks:", len(train_masks))
print("Training labels:", len(train_targets))

if len(train_images) == len(train_masks) == len(train_targets):
    print("✔ Dataset is consistent.")
else:
    print("⚠ Mismatch detected!")

print("Sample image size:", train_images[0].size)
print("Sample mask size:", train_masks[0].size)
print("Sample label:", train_targets[0])

print("Sample image size:", train_images[75].size)
print("Sample mask size:", train_masks[75].size)
print("Sample label:", train_targets[75])

print("Sample image size:", train_images[1000].size)
print("Sample mask size:", train_masks[1000].size)
print("Sample label:", train_targets[1000])


===== DATASET EXPLORATION =====
Training images: 1412
Training masks: 1412
Training labels: 1412
✔ Dataset is consistent.
Sample image size: (1024, 1097)
Sample mask size: (1024, 1097)
Sample label: Triple negative
Sample image size: (1024, 1515)
Sample mask size: (1024, 1515)
Sample label: HER2(+)
Sample image size: (1024, 1582)
Sample mask size: (1024, 1582)
Sample label: Luminal A
